# 正则化和模型选择 Regularization and Model Selection

设想现在对于一个学习问题，需要从一组不同的模型中进行挑选。比如多元回归模型 $h_\theta(x)=g(\theta_0+\theta_1x+\theta_2x^2+\cdots+\theta_kx^k)$，如何自动地确定 $k$ 的取值，从而在偏差和方差之间达到较好的权衡？或者对于局部加权线性回归，如何确定带宽 $\tau$ 的值，以及对于 $\ell_1$ 正则化的支持向量机，如何确定参数 $C$ 的值？

为了方便后续的讨论，统一假定有一组有限数量的模型集合 $\mathcal{M}=\{M_1,\cdots,M_d\}$。（推广到无限数量的集合也非常容易，比如对于局部加权线性模型的带宽 $\tau$，其取值范围为 $\mathbb{R}^+$，只需要将 $\tau$ 离散化，考虑有限的若干个值即可。更一般地，这里讨论的绝大多数算法，都可以看做在模型空间范围内的优化搜索问题）

本节包括以下内容：
1. 交叉验证 Cross validation
2. 特征选择 Feature selection
3. 贝叶斯统计学和正则化 Bayesian statistics and regularization

### 1. 交叉验证 Cross Validation

设想有训练集 $S$。回顾经验风险最小化，一个直观的模型选择过程如下：
1. 对于每一个模型 $M_i$，用 $S$ 进行训练，得到相应的假设函数 $h_i$。
2. 挑选训练误差最小的假设函数。

这个算法的表现可能会很差。考虑多元线性回归，模型的阶越高，它对训练集 $S$ 的拟合情况就越好，从而可以得到越低的训练误差。因此，上面这个方法，总是会挑选出高方差、高阶的多元模型。

下面是 **hold-out 交叉验证**（也称简单交叉验证）的思路：
1. 随机将 $S$ 分为 $S_{train}$（例如大约70%的数据）和 $S_{cv}$（剩下的30%）。这里，$S_{cv}$ 称作hold-out交叉验证集。
2. 对于每一个模型 $M_i$，仅使用 $S_{train}$ 进行训练，得到相应的假设函数 $h_i$。
3. 从 $h_i$ 中挑选对hold-out交叉验证集误差 $\hat{\epsilon}_{cv}(h_i)$ 最小的假设函数。

对未用来训练数据的 $S_{cv}$ 计算的交叉验证集误差，是泛化误差的一个更好的估计量。通常，在hold-out交叉验证中会保留四分之一到三分之一的数据，30%是最常见的选择。

hold-out交叉验证还有一个可选的步骤，在上述流程完成后，可以用模型 $M_i$ 针对整个训练集 $S$ 重新训练。（通常这会训练出稍好的模型，不过也有例外，比如学习算法非常容易收到初始条件或初始数据影响的情况，这时 $M_i$ 在 $S_{train}$ 上表现良好，并不一定也会在 $S_{cv}$ 上表现良好，这种情况最好不要执行这个重新训练的过程）

hold-out交叉验证的一个劣势在于，它“浪费”了30%的数据。即便最后使用整个训练集重新训练了模型，看上去模型挑选的过程还是只针对 $0.7m$ 的训练样本，而不是所有 $m$ 个样本，因为我们的测试的模型只使用了 $0.7m$ 的数据。当数据量非常大的时候，这通常没什么问题，但数据量小的时候，可能就需要更好的策略。

**k折交叉验证**，每次训练时，都只保留更少的数据：
1. 随机将 $S$ 分为 $k$ 个不相交的子集，每个子集包含 $m/k$ 个训练样本。记做 $S_1,\cdots,S_k$。
2. 对于每一个模型 $M_i$：对 $j=1,\cdots,k$，使用 $S_1 \cup \cdots \cup S_{j-1} \cup S_{j+1} \cup \cdots \cup S_k$（即对训练集中除去 $S_j$ 的部分进行训练），得到假设函数 $h_{ij}$。在 $S_j$ 上测试，得到 $\hat{\epsilon}_{S_{j}}(h_{ij})$。最终 $M_i$ 的泛化误差估计量表示为 $\hat{\epsilon}_{S_{j}}(h_{ij})$ 的平均值。
3. 挑选预计泛化误差最小的模型 $M_i$，用整个训练集 $S$ 重新训练，得到最终的假设函数。

最常见的选择是令 $k=10$。由于每次训练时保留的数据量更小，而每个模型都需要训练 $k$ 次，k折交叉验证的计算开销会比hold-out交叉验证更大。

尽管 $k=10$ 是最常用的，但当数据量非常小的时候，有时也会采用 $k=m$ 来保证每次训练尽可能保留最少的数据用于验证。这种特殊情况的k折验证，也叫作**留一交叉验证 leave-one-out cross validation**。

最后，尽管这里介绍交叉验证用于模型选择，实际上，交叉验证也可以用来做单个模型的效果评估。